In [1]:
%load_ext autoreload
%autoreload 2

import qubx
%qubxd

from pathlib import Path
from qubx import logger, QubxLogConfig
from qubx.utils.runner.runner import run_strategy, StrategyConfig, AccountConfigurationManager, ExchangeConfig, LoggingConfig
from qubx.core.interfaces import IStrategy, IStrategyContext, BaseErrorEvent

QubxLogConfig.set_log_level("INFO")


⠀⠀⡰⡖⠒⠒⢒⢦⠀⠀   
⠀⢠⠃⠈⢆⣀⣎⣀⣱⡀  QUBX | Quantitative Backtesting Environment 
⠀⢳⠒⠒⡞⠚⡄⠀⡰⠁         (c) 2025, ver. 0.6.5
⠀⠀⠱⣜⣀⣀⣈⣦⠃⠀⠀⠀ 
        


In [6]:
class TestStrategy(IStrategy):
    def on_error(self, ctx: IStrategyContext, error: BaseErrorEvent) -> None:
        logger.error(f"Error: {error}")

    def on_stop(self, ctx: IStrategyContext):
        logger.info("Stopping test strategy")


ctx = run_strategy(
    config=StrategyConfig(
        name="TestStrategy",
        strategy=TestStrategy,
        exchanges={
            "BINANCE.PM": ExchangeConfig(
                connector="ccxt",
                universe=["ETHUSDT"],
                params={"enable_price_match": True, "price_match_ticks": 5},
            )
        },
        logging=LoggingConfig(
            logger="InMemoryLogsWriter",
            position_interval="10s",
            portfolio_interval="1m",
            heartbeat_interval="10m",
        )
    ),
    account_manager=AccountConfigurationManager(
        Path("~/releases/R_MM_olereon/accounts.toml").expanduser()
    ),
    paper=False,
)

2025-03-21 14:43:55.961 [ℹ️] Initialized BINANCE.PM
2025-03-21 14:43:55.990 [ℹ️] - Strategy: TestStrategy
- Mode: live
- Parameters: {}
2025-03-21 14:43:55.994 [ℹ️] Waiting for account polling tasks to be initialized
2025-03-21 14:43:57.786 [ℹ️] balance polling task has been initialized
2025-03-21 14:43:58.602 [ℹ️] position polling task has been initialized
2025-03-21 14:43:58.783 [ℹ️] Account polling tasks have been initialized
2025-03-21 14:43:58.785 [ℹ️] Listening to executions
2025-03-21 14:43:58.786 [ℹ️] [StrategyContext] :: Start processing market data
2025-03-21 14:43:58.787 [ℹ️] [StrategyContext] :: strategy is started in thread
2025-03-21 14:43:58.787 [ℹ️] Listening to ETH/USDT:USDT orderbook


2025-03-21 14:44:00.611 [ℹ️] Heartbeat at 2025-03-21T14:40:00
2025-03-21 14:44:59.783 [ℹ️] open_orders polling task has been initialized
2025-03-21 14:45:20.969 [❌] (::_create_order) buy 0.002 limit for ETHUSDT exception : binanceusdm {"code":-4164,"msg":"Order's notional must be greater than 20 (unless you choose reduce only)"}
2025-03-21 14:45:20.980 [❌] Traceback (most recent call last):
  File "/home/yuriy/devs/Qubx/src/qubx/connectors/ccxt/broker.py", line 259, in _create_order
    r = await self._exchange.create_order(
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/yuriy/.cache/pypoetry/virtualenvs/qubx-QjLytwEC-py3.12/lib/python3.12/site-packages/ccxt/async_support/binance.py", line 6201, in create_order
    response = await self.papiPostUmOrder(request)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/yuriy/.cache/pypoetry/virtualenvs/qubx-QjLytwEC-py3.12/lib/python3.12/site-packages/ccxt/async_support/binance.py", line 11500, in request
    response =

In [7]:
ctx.get_balances()

{'USDT': AssetBalance(free=19993.91813323, locked=0.0, total=19993.91813323)}

In [8]:
instr = ctx.instruments[0]
instr

BINANCE.UM:SWAP:ETHUSDT

In [10]:
instr.min_size

0.001

In [16]:
q = ctx.quote(instr)
assert q is not None
print(q)

# ctx.trade_async(instr, amount=0.02, price=q.bid + instr.tick_size, time_in_force="gtx")
ctx.trade_async(instr, amount=-0.02, price=q.ask - instr.tick_size, time_in_force="gtx")
# ctx.trade(instr, amount=0.002, price=q.bid + 500, time_in_force="gtx")

[2025-03-21T14:49:49.867000000]	1946.70000 (42.3) | 1946.71000 (418.1)


In [7]:
ctx.get_orders()

{}

In [24]:
ctx.cancel_order("632039574280")

In [5]:
ctx.stop()